In [1]:
#import dependencies
import pandas as pd
import numpy as np

In [2]:
#Import CSV files
cost_csvpath = "../Resources/cost_of_living.csv"
cities_csvpath="../Resources/CitiesWGeolocation.csv"

cost_raw_df = pd.read_csv(cost_csvpath,index_col=0)
cities_df = pd.read_csv(cities_csvpath,index_col=0)


In [3]:
cities_df["Country"] = cities_df["Country"].str.strip()
cities_df["City"] = cities_df["City"].str.strip()

In [4]:
# Documentation from Kaggle Dataset explaining the names of the columns. Filtered to only show columns we want to keep
# city	Name of the city
# country	Name of the country
# x2	Meal for 2 People, Mid-range Restaurant, Three-course (USD)
# x28	One-way Ticket (Local Transport) (USD)
# x30	Taxi Start (Normal Tariff) (USD)
# x31	Taxi 1km (Normal Tariff) (USD)

In [12]:
cost_df = cost_raw_df[["city","country","x2","x28","x30","x31"]]
cost_df = cost_df.rename(columns={"x2":"meal","x28":"ticket","x30":"taxi_start","x31":"taxi_1km"})
cost_df = cost_df.replace(np.nan,0)
cost_df

,city,country,meal,ticket,taxi_start,taxi_1km
0,Delhi,India,22.04,0.49,0.61,0.22
1,Shanghai,China,40.51,0.56,2.10,0.42
2,Jakarta,Indonesia,22.25,0.22,0.41,0.29
3,Manila,Philippines,27.40,0.53,0.71,0.24
4,Seoul,South Korea,52.77,0.98,2.86,0.90
...,...,...,...,...,...,...
4869,Peterborough,Australia,0.00,0.00,0.00,0.00
4870,Georgetown,Australia,0.00,0.00,0.00,0.00
4871,Ixtapa Zihuatanejo,Mexico,31.13,0.00,4.15,1.04
4872,Iqaluit,Canada,74.61,6.33,5.96,0.00


In [6]:
cities_df["meal"] = 0
cities_df["ticket"] = 0
cities_df["taxi_start"] = 0
cities_df["taxi_1km"] = 0

In [7]:
cities_df.at[0,"meal"]= 5
cities_df.reset_index(inplace=True)
cities_df.replace(np.nan,0)
cities_df

,index,City,State/Province,Country,Latitude,Longitude,meal,ticket,taxi_start,taxi_1km
0,0,Lexington,Kentucky,United States,38.046407,-84.497039,5,0,0,0
1,1,San Diego,California,United States,32.717420,-117.162773,0,0,0,0
2,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871,0,0,0,0
3,3,Park City,Utah,United States,40.646092,-111.497996,0,0,0,0
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
215,232,Beirut,NaN,Lebanon,33.895920,35.478430,0,0,0,0
216,233,Zurich,NaN,Switzerland,47.374449,8.541042,0,0,0,0
217,234,Geneva,NaN,Switzerland,46.201756,6.146601,0,0,0,0
218,235,Valletta,NaN,Malta,35.898982,14.513676,0,0,0,0


In [15]:
for i in range(len(cities_df)):
    if len(cost_df.loc[(cost_df['city'] == cities_df['City'][i]) & (cost_df['country'] == cities_df['Country'][i])]) == 1:
        result = cost_df.loc[(cost_df['city'] == cities_df['City'][i]) & (cost_df['country'] == cities_df['Country'][i])]
        cities_df.at[i,"meal"] = result['meal']
        cities_df.at[i,"ticket"] = result["ticket"]
        cities_df.at[i,"taxi_start"] = result["taxi_start"]
        cities_df.at[i,"meal"] = result["taxi_1km"]
        cities_df.at[i,"data_quality"] = 1
    elif len(cost_df.loc[(cost_df['city'] == cities_df['City'][i]) & (cost_df['country'] == cities_df['Country'][i])]) == 0:
        if len(cost_df.loc[(cost_df['country'] == cities_df['Country'][i])]) >=1:
            result = cost_df.loc[(cost_df['country'] == cities_df['Country'][i])]
            cities_df.at[i,"meal"] = result["meal"].mean()
            cities_df.at[i,"meal"] = result["ticket"].mean()
            cities_df.at[i,"meal"] = result["taxi_start"].mean()
            cities_df.at[i,"meal"] = result["taxi_1km"].mean()
            cities_df.at[i,"data_quality"] = 2

In [17]:
cities_df[cities_df["data_quality"].isna()]

,index,City,State/Province,Country,Latitude,Longitude,meal,ticket,taxi_start,taxi_1km,data_quality
2,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871,0,0,0,0,NaN
13,13,Antigua and Barbuda​,NaN,Antigua and Barbuda​,17.223472,-61.955461,0,0,0,0,NaN
16,16,Turks and Caicos Islands​,NaN,Turks and Caicos Islands​,21.721746,-71.552781,0,0,0,0,NaN
27,29,Mauritius​,NaN,Mauritius​,-20.275945,57.570357,0,0,0,0,NaN
29,31,Cayman Islands​,NaN,Cayman Islands​,19.703182,-79.917463,0,0,0,0,NaN
30,32,Seychelles​,NaN,Seychelles​,-4.657498,55.454015,0,0,0,0,NaN
42,47,Barbados​,NaN,Barbados​,13.150033,-59.525031,0,0,0,0,NaN
49,56,Jamaica​,NaN,Jamaica​,18.185051,-77.394769,0,0,0,0,NaN
50,58,Aruba,Kingdom of the Netherlands,Kingdom of the Netherlands,12.517566,-69.981864,0,0,0,0,NaN
75,86,Fiji​,NaN,Fiji​,-18.123970,179.012274,0,0,0,0,NaN
